In [156]:
'''
We will need the stopwords from NLTK and spacy’s en model for text pre-processing.
Later, we will be using the spacy model for lemmatization.

Lemmatization is nothing but converting a word to its root word.
For example: the lemma of the word ‘machines’ is ‘machine’.

Likewise, ‘walking’ –> ‘walk’, ‘mice’ –> ‘mouse’ and so on.

'''

# Run in python console
import nltk
nltk.download('stopwords')

# nltk.download('stopwords') # You already did this in a previous lab

[nltk_data] Downloading package stopwords to C:\Users\Yong
[nltk_data]     Jie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [157]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [158]:
# Gensim (Topic Modeling Pacakge)
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [159]:
# spacy for lemmatization
import spacy

In [160]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [161]:
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [162]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['scanteaksg','scanteak'])

In [163]:
'''
We will be using the 20-Newsgroups dataset for this exercise.
This version of the dataset contains about 11k newsgroups posts from 
20 different topics. This is available as newsgroups.json.
'''
# Import Dataset
df = pd.read_csv('./scanteaksg.csv', encoding='utf-8')

df.head()

,date,username,text
0,2013-12-23 01:24:35,phumyatmon,Cute :heart_suit::heart_suit: :)
1,2014-12-16 10:12:58,giftsnsuchsg,:) my pleasure! So want to go for it but l m s...
2,2014-12-16 09:59:05,uniquegrabs,@giftsnsuchsg Thank you so much for your well ...
3,2014-12-16 08:34:55,giftsnsuchsg,All the best!
4,2014-12-16 02:09:05,uniquegrabs,#cardmaking #christmas #handmadecard #workshop...


In [164]:
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())





[nltk_data] Downloading package words to C:\Users\Yong
[nltk_data]     Jie\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [165]:
'''
As you can see there are many emails, newline and extra spaces that is quite distracting.
Let’s get rid of them using regular expressions.
'''
# Convert to list
data = df.text.values.tolist()
new_data = []

# Remove Emails
data = [re.sub(r'[^\x00-\x7F]+', '', sent) for sent in data]

# # Remove new line characters
# data = [re.sub('\s+', ' ', sent) for sent in data]

# # Remove distracting single quotes
# data = [re.sub("\'", "", sent) for sent in data]

        
    

print(data[:20])

['Cute :heart_suit::heart_suit: :)', ':) my pleasure! So want to go for it but l m so bz during this season.....', '@giftsnsuchsg Thank you so much for your well wishes, Mrs Chan! :)', 'All the best!', '#cardmaking #christmas #handmadecard #workshop #scanteaksg #papercrafts #crafts #stationery #uniquegrabs', 'hermeswechat:aifeihermes', '#trickeye #chistmas #snowglobe', '#scanteaksmile', 'wow!', 'Hi! Looks like we will be doing business in future haha!', 'exactly my thoughts. pink polka dot chair will be most ideal haha', '@cherrychoo cannot help thinking of you and polka dots. Though you would have preferred pink instead I think.', 'nice! polka dots! :smiling_face_with_heart-eyes:', ':thumbs_up_medium-light_skin_tone:', 'Amazing :clapping_hands_medium_skin_tone::clapping_hands_medium_skin_tone::clapping_hands_medium_skin_tone:', 'What do you think of this? @stephlimmss', '@eternalgluck zhong kan bu zhong yong', '@hellonoraa the coffee table though', 'Love it!', 'i wan @hpu3m @theminilo

In [166]:
'''
After removing the emails and extra spaces, the text still looks messy.
It is not ready for the LDA to consume. You need to break down each sentence
into a list of words through tokenization, while clearing up all the messy 
text in the process.

Let’s tokenize each sentence into a list of words, removing punctuations and 
unnecessary characters altogether.

Gensim’s simple_preprocess() is great for this.
Additionally I have set deacc=True to remove the punctuations.
'''
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[0])
for list_of_words in data_words:
    for word in list_of_words:
        if word.lower() in words or not word.isalpha():
            list_of_words.remove(word)
    
print(data_words[:80])

['cute', 'heart_suit', 'heart_suit']
[['heart_suit'], ['pleasure', 'want', 'go', 'it', 'so', 'bz', 'this'], ['giftsnsuchsg', 'you', 'much', 'your', 'wishes', 'mrs', 'chan'], ['the'], ['christmas', 'handmadecard', 'scanteaksg', 'papercrafts', 'crafts', 'uniquegrabs'], ['hermeswechat', 'aifeihermes'], ['trickeye', 'chistmas', 'snowglobe'], ['scanteaksmile'], [], ['looks', 'we', 'be', 'business', 'future', 'haha'], ['my', 'thoughts', 'polka', 'chair', 'be', 'ideal', 'haha'], ['cherrychoo', 'help', 'of', 'and', 'dots', 'you', 'have', 'pink', 'think'], ['polka', 'dots', 'eyes'], [], [], ['do', 'think', 'this', 'stephlimmss'], ['eternalgluck', 'zhong', 'bu', 'zhong', 'yong'], ['hellonoraa', 'coffee', 'though'], ['it'], ['hpu', 'theminiloft'], [], ['love'], ['sallytumultuous', 'pleasure', 'ours', 're', 'you', 'happy', 'andy', 'and', 'definitely', 'on', 'compliments', 'him', 'great', 'hugging_face'], ['you', 'the', 'scanteaksg', 'thanks', 'the', 'staff', 'park', 'forgive', 'if', 'got', 'name',

In [167]:
data_words = [x for x in data_words if x]

print(data_words[0:20])

[['heart_suit'], ['pleasure', 'want', 'go', 'it', 'so', 'bz', 'this'], ['giftsnsuchsg', 'you', 'much', 'your', 'wishes', 'mrs', 'chan'], ['the'], ['christmas', 'handmadecard', 'scanteaksg', 'papercrafts', 'crafts', 'uniquegrabs'], ['hermeswechat', 'aifeihermes'], ['trickeye', 'chistmas', 'snowglobe'], ['scanteaksmile'], ['looks', 'we', 'be', 'business', 'future', 'haha'], ['my', 'thoughts', 'polka', 'chair', 'be', 'ideal', 'haha'], ['cherrychoo', 'help', 'of', 'and', 'dots', 'you', 'have', 'pink', 'think'], ['polka', 'dots', 'eyes'], ['do', 'think', 'this', 'stephlimmss'], ['eternalgluck', 'zhong', 'bu', 'zhong', 'yong'], ['hellonoraa', 'coffee', 'though'], ['it'], ['hpu', 'theminiloft'], ['love'], ['sallytumultuous', 'pleasure', 'ours', 're', 'you', 'happy', 'andy', 'and', 'definitely', 'on', 'compliments', 'him', 'great', 'hugging_face'], ['you', 'the', 'scanteaksg', 'thanks', 'the', 'staff', 'park', 'forgive', 'if', 'got', 'name', 'up', 'believe', 'name', 'andy', 'him', 'say']]


In [168]:
'''
Bigrams are two words frequently occurring together in the document.
Trigrams are 3 words frequently occurring.

Some examples in our example are: ‘front_bumper’, ‘oil_leak’, ‘maryland_college_park’ etc.

Gensim’s Phrases model can build and implement the bigrams, trigrams, 
quadgrams and more.

The two important arguments to Phrases are min_count and threshold.
The higher the values of these param, the harder it is for words to be combined to bigrams.
'''
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['heart_suit']


In [169]:
'''
The bigrams model is ready. Let’s define the functions to remove the stopwords, 
make bigrams and lemmatization and call them sequentially.
'''
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [170]:
'''
Let’s call the functions in order.
'''
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['heart_suit']]


In [171]:
'''
The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus.
Let’s create them.
'''
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1)]]


In [172]:
'''
Gensim creates a unique id for each word in the document.
The produced corpus shown above is a mapping of (word_id, word_frequency).

For example, (0, 1) above implies, word id 0 occurs once in the first document.

Likewise, word id 1 occurs twice and so on.

This is used as the input by the LDA model.

If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

id2word[0]
'addition'

Or, you can see a human-readable form of the corpus itself.

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('maryland_college', 1),
  (..truncated..)]]


Alright, without digressing further let’s jump back on track with the next step:

Building the topic model.
'''

"\nGensim creates a unique id for each word in the document.\nThe produced corpus shown above is a mapping of (word_id, word_frequency).\n\nFor example, (0, 1) above implies, word id 0 occurs once in the first document.\n\nLikewise, word id 1 occurs twice and so on.\n\nThis is used as the input by the LDA model.\n\nIf you want to see what word a given id corresponds to, pass the id as a key to the dictionary.\n\nid2word[0]\n'addition'\n\nOr, you can see a human-readable form of the corpus itself.\n\n# Human readable format of corpus (term-frequency)\n[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]\n\n[[('addition', 1),\n  ('anyone', 2),\n  ('body', 1),\n  ('bricklin', 1),\n  ('bring', 1),\n  ('call', 1),\n  ('car', 5),\n  ('could', 1),\n  ('day', 1),\n  ('door', 2),\n  ('early', 1),\n  ('engine', 1),\n  ('enlighten', 1),\n  ('front_bumper', 1),\n  ('maryland_college', 1),\n  (..truncated..)]]\n\n\nAlright, without digressing further let’s jump back on track with the nex

In [173]:
'''
We have everything required to train the LDA model.
In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics.
According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk.

update_every determines how often the model parameters should be updated and 
passes is the total number of training passes.
'''
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [174]:
'''
The above LDA model is built with 20 different topics where each topic is a 
combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of 
each keyword using lda_model.print_topics() as shown next.
'''
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

'''
The output will look like this...


[(0,
  '0.016*"car" + 0.014*"power" + 0.010*"light" + 0.009*"drive" + 0.007*"mount" '
  '+ 0.007*"controller" + 0.007*"cool" + 0.007*"engine" + 0.007*"back" + '
  '0.006*"turn"'),
 (1,
  '0.072*"line" + 0.066*"organization" + 0.037*"write" + 0.032*"article" + '
  '0.028*"university" + 0.027*"nntp_post" + 0.026*"host" + 0.016*"reply" + '
  '0.014*"get" + 0.013*"thank"'),
 (2,
  '0.017*"patient" + 0.011*"study" + 0.010*"slave" + 0.009*"wing" + '
  '0.009*"disease" + 0.008*"food" + 0.008*"eat" + 0.008*"pain" + '
  '0.007*"treatment" + 0.007*"syndrome"'),
 (3,
  '0.013*"key" + 0.009*"use" + 0.009*"may" + 0.007*"public" + 0.007*"system" + '
  '0.007*"order" + 0.007*"government" + 0.006*"state" + 0.006*"provide" + '
  '0.006*"law"'),
  
  so on...



How to interpret this?

Topic 0 is a represented as _

0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” 
+ 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” 
+ 0.007“back” + ‘0.006“turn”.

It means the top 10 keywords that contribute to this topic are:
‘car’, ‘power’, ‘light’.. and so on
and the weight of ‘car’ on topic 0 is 0.016.

--> The weights reflect how important a keyword is to that topic.

Looking at these keywords, can you guess what this topic could be?
You may summarise it either are ‘cars’ or ‘automobiles’.

Likewise, can you go through the remaining topic keywords and judge what the topic is?
https://www.machinelearningplus.com/wp-content/uploads/2018/03/Inferring-Topic-from-Keywords.png
'''

[(0,
  '0.142*"eye" + 0.022*"cantik" + 0.020*"sparkle" + 0.012*"red_heart" + '
  '0.012*"interior" + 0.010*"hugging_face" + 0.010*"much" + 0.009*"pm" + '
  '0.008*"furniture" + 0.008*"want"'),
 (1,
  '0.041*"sghome" + 0.041*"feelathome" + 0.040*"furniturestyle" + '
  '0.040*"sginterior" + 0.040*"homeinspiration" + 0.040*"homeinspo" + '
  '0.040*"homestyle" + 0.040*"homestyling" + 0.040*"interiorsg" + '
  '0.040*"sghomedecor"'),
 (2,
  '0.031*"love" + 0.023*"look" + 0.019*"year" + 0.015*"think" + 0.013*"sofa" + '
  '0.012*"heart_suit" + 0.012*"cover" + 0.011*"come" + 0.009*"thank" + '
  '0.009*"machine"')]


'\nThe output will look like this...\n\n\n[(0,\n  \'0.016*"car" + 0.014*"power" + 0.010*"light" + 0.009*"drive" + 0.007*"mount" \'\n  \'+ 0.007*"controller" + 0.007*"cool" + 0.007*"engine" + 0.007*"back" + \'\n  \'0.006*"turn"\'),\n (1,\n  \'0.072*"line" + 0.066*"organization" + 0.037*"write" + 0.032*"article" + \'\n  \'0.028*"university" + 0.027*"nntp_post" + 0.026*"host" + 0.016*"reply" + \'\n  \'0.014*"get" + 0.013*"thank"\'),\n (2,\n  \'0.017*"patient" + 0.011*"study" + 0.010*"slave" + 0.009*"wing" + \'\n  \'0.009*"disease" + 0.008*"food" + 0.008*"eat" + 0.008*"pain" + \'\n  \'0.007*"treatment" + 0.007*"syndrome"\'),\n (3,\n  \'0.013*"key" + 0.009*"use" + 0.009*"may" + 0.007*"public" + 0.007*"system" + \'\n  \'0.007*"order" + 0.007*"government" + 0.006*"state" + 0.006*"provide" + \'\n  \'0.006*"law"\'),\n  \n  so on...\n\n\n\nHow to interpret this?\n\nTopic 0 is a represented as _\n\n0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” \n+ 0.007“mount” + 0.007“controller” + 0.00

In [175]:
'''
Model perplexity and topic coherence provide a convenient measure to judge 
how good a given topic model is. In my experience, topic coherence score, 
in particular, has been more helpful.
'''
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

'''
Perplexity:  -8.86067503009

Coherence Score:  0.532947587081


There you have a coherence score of 0.53.
'''


Perplexity:  -5.430642340072366

Coherence Score:  0.5517435530091962


'\nPerplexity:  -8.86067503009\n\nCoherence Score:  0.532947587081\n\n\nThere you have a coherence score of 0.53.\n'

In [176]:
'''
Now that the LDA model is built, the next step is to examine the produced 
topics and the associated keywords. There is no better tool than pyLDAvis package’s 
interactive chart and is designed to work well with jupyter notebooks.
'''
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.220756  0.008781       1        1  57.513566
2     -0.097144 -0.114250       2        1  21.711565
0     -0.123612  0.105469       3        1  20.774869, topic_info=               Term       Freq      Total Category  logprob  loglift
34              eye  25.000000  25.000000  Default  30.0000  30.0000
44             love   6.000000   6.000000  Default  29.0000  29.0000
230          sghome  20.000000  20.000000  Default  28.0000  28.0000
86       feelathome  20.000000  20.000000  Default  27.0000  27.0000
284  furniturestyle  19.000000  19.000000  Default  26.0000  26.0000
..              ...        ...        ...      ...      ...      ...
358        designer   0.896319   1.344763   Topic3  -5.2950   1.1657
50     hugging_face   1.870478   2.837226   Topic3  -4.5593   1.1548
8              much   1.777854   2.923968   Topic3  -4.6101   1.0739
142       furniture   1.383563   2.350197   Topic3  -4.8608   1.0416
48            great   1.171754   2.138425   Topic3  -5.0270   0.9699

[133 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
276       3  0.743635     also
95        3  0.545917     awww
204       3  0.743635  blossom
327       1  0.858896      brp
155       3  0.905959   cantik
...     ...       ...      ...
112       3  0.390528  website
113       1  0.964503      www
72        2  0.766375     year
232       3  0.743635      yet
38        2  0.594227    zhong

[115 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [177]:
'''
So how to infer pyLDAvis’s output?

Each bubble on the left-hand side plot represents a topic. 
The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered 
throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, 
small sized bubbles clustered in one region of the chart.

Alright, if you move the cursor over one of the bubbles, 
the words and bars on the right-hand side will update. 
These words are the salient keywords that form the selected topic.

We have successfully built a good looking topic model.
'''

'\nSo how to infer pyLDAvis’s output?\n\nEach bubble on the left-hand side plot represents a topic. \nThe larger the bubble, the more prevalent is that topic.\n\nA good topic model will have fairly big, non-overlapping bubbles scattered \nthroughout the chart instead of being clustered in one quadrant.\n\nA model with too many topics, will typically have many overlaps, \nsmall sized bubbles clustered in one region of the chart.\n\nAlright, if you move the cursor over one of the bubbles, \nthe words and bars on the right-hand side will update. \nThese words are the salient keywords that form the selected topic.\n\nWe have successfully built a good looking topic model.\n'

In [178]:
pyLDAvis.save_html(vis, 'scanteaksg_lda.html')